In [4]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.21.230828.2-preview"
#r "nuget: SkiaSharp"

Installed Packages Microsoft.SemanticKernel, 0.21.230828.2-preview SkiaSharp, 2.88.6

Loading extensions from `C:\Users\Ray\.nuget\packages\skiasharp\2.88.6\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

In [2]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [9]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ImageGeneration; 
using Microsoft.SemanticKernel.AI.Embeddings;
using Microsoft.SemanticKernel.AI.Embeddings.VectorOperations;

In [10]:
// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if(useAzureOpenAI)
{
    builder.WithAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
    builder.WithAzureTextCompletionService("gpt-35-turbo", azureEndpoint, apiKey);
    builder.WithAzureOpenAIImageGenerationService(azureEndpoint, apiKey);
}
else
{
    builder.WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", orgId);
    builder.WithOpenAITextCompletionService("gpt-3.5-turbo", orgId);
    builder.WithOpenAIImageGenerationService(apiKey, orgId);
}
   
var kernel = builder.Build();

Error: System.ArgumentException: The value cannot be an empty string or composed entirely of whitespace. (Parameter 'apiKey')
   at Microsoft.SemanticKernel.Diagnostics.Verify.ThrowArgumentWhiteSpaceException(String paramName)
   at Microsoft.SemanticKernel.Connectors.AI.OpenAI.AzureSdk.OpenAIClientBase..ctor(String modelId, String apiKey, String organization, HttpClient httpClient, ILoggerFactory loggerFactory)
   at Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextEmbedding.OpenAITextEmbeddingGeneration..ctor(String modelId, String apiKey, String organization, HttpClient httpClient, ILoggerFactory loggerFactory)
   at Microsoft.SemanticKernel.OpenAIKernelBuilderExtensions.<>c__DisplayClass6_0.<WithOpenAITextEmbeddingGenerationService>b__0(ILoggerFactory loggerFactory, KernelConfig config)
   at Microsoft.SemanticKernel.KernelBuilder.<>c__DisplayClass21_0`1.<WithAIService>b__0()
   at Microsoft.SemanticKernel.Services.NamedServiceProvider`1.GetService[T](String name)
   at Mic

In [ ]:
// Get AI service instance used to generate images
var dallE = kernel.GetService<IImageGeneration>();

// Get AI service instance used to extract embedding from a text
var textEmbedding = kernel.GetService<ITextEmbeddingGeneration>();

In [ ]:
// Create a semantic function that generate a random image description.
var genImgDescription = kernel.CreateSemanticFunction(
    "Think about an artificial object correlated to number {{$input}}. " +
    "Describe the image with one detailed sentence. The description cannot contain numbers.", 
    maxTokens: 256, temperature: 1);

var random = new Random().Next(0, 200);
var imageDescription = await genImgDescription.InvokeAsync($"{random}");

// Use DALL-E 2 to generate an image. OpenAI in this case returns a URL (though you can ask to return a base64 image)
var imageUrl = await dallE.GenerateImageAsync(imageDescription.Result.Trim(), 512, 512);

await SkiaUtils.ShowImage(imageUrl, 512, 512);

In [ ]:
var guess = await InteractiveKernel.GetInputAsync("Describe the image in your words");

// Compare user guess with real description and calculate score
var origEmbedding = await textEmbedding.GenerateEmbeddingsAsync(new List<string> { imageDescription.Result} );
var guessEmbedding = await textEmbedding.GenerateEmbeddingsAsync(new List<string> { guess } );
var similarity = origEmbedding.First().AsReadOnlySpan().CosineSimilarity(guessEmbedding.First().AsReadOnlySpan());

Console.WriteLine($"Your description:\n{Utils.WordWrap(guess, 90)}\n");
Console.WriteLine($"Real description:\n{Utils.WordWrap(imageDescription.Result.Trim(), 90)}\n");
Console.WriteLine($"Score: {similarity:0.00}\n\n");

//Uncomment this line to see the URL provided by OpenAI
//Console.WriteLine(imageUrl);